<a href="https://colab.research.google.com/github/dnguyend/ManNullRange/blob/master/tests/positive_semidefinite_symbolic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\newcommand{\KK}{\mathbb{K}}$
$\newcommand{\rN}{\mathrm{N}}$
$\newcommand{\rD}{\mathrm{D}}$
$\newcommand{\cE}{\mathcal{E}}$
$\newcommand{\ft}{\mathfrak{t}}$
$\newcommand{\cEJ}{\cE_{\JJ}}$
$\newcommand{\CTRL}{\textsc{CTRL}}$
$\newcommand{\Herm}[3]{\mathrm{Sym}_{#1, #2, #3}}$
$\newcommand{\AHerm}[3]{\mathrm{Skew}_{#1, #2, #3}}$
$\newcommand{\St}[3]{\mathrm{St}_{#1, #2, #3}}$
$\newcommand{\Sp}[3]{\mathtt{S}^{+}_{#1, #2, #3}}$
$\newcommand{\Sd}[2]{\mathtt{S}^{+}_{#1, #2}}$
$\DeclareMathOperator{\UU}{U}$
$\DeclareMathOperator{\JJ}{J}$
$\DeclareMathOperator{\xtrace}{xtrace}$
# Symbolic calculation for Positive Semidefinite manifold $\Sp{\KK}{p}{n}$ with Riemannian quotient metric, with $\KK$ real or complex.
* Manifold is Stiefel $\St{\KK}{p}{n}$ times positive definite (PD) $\Sd{\KK}{p}$ with the quotient by the unitary group $\UU(\KK, p)$, via a representation $S = YPY^{\ft}$ with $Y\in\St{\KK}{p}{n}, P\in \Sd{\KK}{p}$.
* Horizontal lift is normal to vertical vectors.
* Ambient space $\cE= \KK^{n\times p}\oplus\KK^{p\times p}$
* The tangent space is the nullspace of the operator $\JJ(S)$ is $[\omega_P-\omega_P^{\ft},  \alpha_1 Y^{H} \omega_Y+\beta(\omega_P P^{-1}-\omega_P P^{-1}) ]$ in $\cE_{\JJ} = \AHerm{\KK}{p}{p}\oplus \KK^{p\times p}$
* Operator $\rN(S)$ is $\rN[B, D]=[\beta Y(P^{-1}D - D P^{-1}) + Y_{\perp}B, \alpha_1 D]$ parameterizing the tangent space by $\cE_{\rN} = \KK^{(n-p)\times p}\oplus \Herm{\ft}{\KK}{p}$.


In [1]:
!git clone https://github.com/dnguyend/ManNullRange.git

Cloning into 'ManNullRange'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 125 (delta 83), reused 90 (delta 54), pack-reused 0
Receiving objects: 100% (125/125), 121.34 KiB | 347.00 KiB/s, done.
Resolving deltas: 100% (83/83), done.


In [2]:
from collections import OrderedDict
from sympy import Integer
from IPython.display import display, Math
from ManNullRange.symbolic import SymMat as sm
from ManNullRange.symbolic.SymMat import (
    matrices, stiefels, scalars, t, mat_spfy,
    xtrace, trace, DDR, sym_symb, asym_symb,
    latex_map, mat_latex, simplify_pd_tangent, inv)

def sprint(expr):
    return latex_map(mat_latex(expr), OrderedDict(
        [('fYY', r'f_{YY}'), ('fY', 'f_Y'), ('al', r'\alpha'), ('bt', r'\beta')]))


def pprint(expr):
    display(Math(sprint(expr)))


* Define the symbols. A manifold point consists of $Y, P$ with $S = YPY^{\ft}$. $B, D$ parameterize the tangent space via $N(B, D)$. 

In [3]:
if True:
    # in this method, the ambient is still
    # R(n times p) + Symmetric(p)
    # Horizontal is still
    # al1*t(Y)*omg_Y + bt*omg_P*inv(R*R) - bt*inv(R*R)*omg_P)
    # manifold is on pair (Y, P)
    # Use pair B, D. B size n*(n-p), D size p(p+1)/2
    # Use the embedding
    # omg_Y = bt(Y*(inv(P)*D - D*inv(P))
    # omg_P = al1*D
    # so need to set up a new variable Y_0 and relation YY_0=0
    
    Y, Y0 = stiefels('Y Y0')
    sm.g_cstiefels[Y] = Y0
    sm.g_cstiefels[Y0] = Y
    
    B, eta_Y, eta_P = matrices('B eta_Y eta_P')
    P, D = sym_symb('P D')
    al0, al1, bt = scalars('al0 al1 bt')
    
    def g(Y, P, omg_Y, omg_P):
        return al0*omg_Y+(al1-al0)*Y*t(Y)*omg_Y, bt*inv(P)*omg_P*inv(P)

    def ginv(Y, P, omg_Y, omg_P):
        return 1/al0*omg_Y+(1/al1-1/al0)*Y*t(Y)*omg_Y, 1/bt*P*omg_P*P

    # check that ginv \circ g is id
    e1, e2 = ginv(Y, P, *(g(Y, P, eta_Y, eta_P)))
    e1 = mat_spfy(e1)
    e2 = mat_spfy(e2)
    pprint("\\text{Showing }g \circ g^{-1} \eta = \eta")
    print(e1, e2)


<IPython.core.display.Math object>

eta_Y eta_P


We define base (Frobenius) inner product on the ambient spaces. as sum of two trace expressions. We show the horizontal condition can be derived through $\xtrace$


In [4]:
def base_ambient_inner(omg_Y, omg_P, xi_Y, xi_P):
    return mat_spfy(
        trace(
            mat_spfy(omg_Y * t(xi_Y))) +
        trace(mat_spfy(
            omg_P*t(xi_P))))

def ambient_inner(Y, P, omg_Y, omg_P, xi_Y, xi_P):
    return mat_spfy(
        trace(
            mat_spfy(
                (al0*omg_Y+(al1-al0)*Y*t(Y)*omg_Y) * t(xi_Y))) +
        trace(mat_spfy(
            bt*inv(P)*omg_P*inv(P)*t(xi_P))))

def EN_inner(Y, P, Ba, Da, Bb, Db):
    return trace(
        mat_spfy(Da * Db) + mat_spfy(
            Ba*t(Bb)))

qat = asym_symb('qat')

ipr = ambient_inner(Y, P, eta_Y, eta_P, Y * qat,  P*qat - qat*P)
dqat = mat_spfy(xtrace(ipr, qat))
print("The Horizontal condtion")
pprint(dqat)


The Horizontal condtion


<IPython.core.display.Math object>

In [5]:
def N(Y, P, B, D):
    N_Y = mat_spfy(bt*Y*(inv(P)*D - D*inv(P))) + Y0*B
    N_P = mat_spfy(
        al1*D)

    return N_Y, N_P

def NT(Y, P, omg_Y, omg_P):
    nB, nD = matrices('nB nD')
    ipt = mat_spfy(
        base_ambient_inner(*N(Y, P, nB, nD), omg_Y, omg_P))
    ntB = mat_spfy(xtrace(ipt, nB))
    ntD1 = mat_spfy(xtrace(ipt, nD))
    ntD = mat_spfy(Integer(1)/Integer(2)*(ntD1 + t(ntD1)))
    return ntB, ntD

# check that image of N is horizontal:
print("Check that image of N is horizontal, so inner product with a vertical vector is zero")
pprint(mat_spfy(xtrace(
    mat_spfy(
        ambient_inner(Y, P, *N(Y, P, B, D), Y*qat, P*qat - qat*P)), qat)))
NTe_B, NTe_D = NT(Y, P, eta_Y, eta_P)

display(Math('\\text{ Formula for } N^Te_B: %s' % sprint(NTe_B)))
display(Math('\\text{ Formula for } N^Te_D: %s '% sprint(NTe_D)))

gN = g(Y, P, *N(Y, P, B, D))
gN_Y = mat_spfy(gN[0])
gN_P = mat_spfy(gN[1])
display(Math('\\text{ Formula for } (gN)_Y: %s' % sprint(gN_Y)))
display(Math('\\text{ Formula for } (gN)_P: %s' % sprint(gN_P)))

NTg_B, NTg_D = NT(Y, P, *g(Y, P, eta_Y, eta_P))
display(Math('\\text{ Formula for } (N^T)_B: %s' % sprint(NTg_B)))
display(Math('\\text{ Formula for } (N^T)_D: %s' % sprint(NTg_D)))

NTgN_B, NTgN_D = NT(Y, P, *gN)

display(Math('\\text{ Formula for } (N^TgN)_B: %s' % sprint(NTgN_B)))
display(Math('\\text{ Formula for } (N^TgN)_D: %s' % sprint(NTgN_D)))


Check that image of N is horizontal, so inner product with a vertical vector is zero


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

* Calculation of the Christoffel metric term:

In [6]:
def sym(x):
        return mat_spfy(
            Integer(1)/Integer(2)*(x + t(x)))
xi_Y, xi_P, phi_Y, phi_P = matrices('xi_Y xi_P phi_Y phi_P')
gYPeta = g(Y, P, eta_Y, eta_P)
Dgxieta_Y = DDR(gYPeta[0], Y, xi_Y)
Dgxieta_P = DDR(gYPeta[1], P, xi_P)

gYPxi = g(Y, P, xi_Y, xi_P)
Dgetaxi_Y = DDR(gYPxi[0], Y, eta_Y)
Dgetaxi_P = DDR(gYPxi[1], P, eta_P)

Dgxiphi_Y = DDR(gYPeta[0], Y, phi_Y)
Dgxiphi_P = DDR(gYPeta[1], P, phi_P)

tr3 = mat_spfy(
    ambient_inner(Y, P, Dgxiphi_Y, Dgxiphi_P, eta_Y, eta_P))
xcross_Y = xtrace(tr3, phi_Y)
xcross_P = xtrace(tr3, phi_P)
    
K_Y = (Integer(1)/Integer(2))*(Dgxieta_Y + Dgetaxi_Y - xcross_Y)
K_P = (Integer(1)/Integer(2))*(Dgxieta_P + Dgetaxi_P - xcross_P)

display(Math('\\text{ Formula for } K_Y: %s' % sprint(K_Y)))
display(Math('\\text{ Formula for } K_P: %s' % sprint(K_P)))



<IPython.core.display.Math object>

<IPython.core.display.Math object>

Now run the formula on the nullspace approach:

In [7]:
a_P = asym_symb('a_P')
a_YP, eta_Y, eta_P = matrices('a_YP eta_Y eta_P')
aYPev = sym(a_YP)

def EJ_inner(Y, P, a_P, a_YP, b_P, b_YP):
    return trace(
        mat_spfy(-a_P * b_P) + mat_spfy(
            a_YP*t(b_YP)))
def J(Y, P, omg_Y, omg_P):
    J_P = mat_spfy(omg_P - t(omg_P))
    J_YP = mat_spfy(
        al1*t(Y)*omg_Y + bt*omg_P*inv(P) - bt*inv(P)*omg_P)

    return J_P, J_YP

def JT(Y, P, a_P, a_YP):
    dY, dP = matrices('dY dP')
    ipt = mat_spfy(
        EJ_inner(Y, P, *J(Y, P, dY, dP), a_P, a_YP))
    jty = mat_spfy(xtrace(ipt, dY))
    jtp = mat_spfy(xtrace(ipt, dP))
    return jty, jtp

JTa_Y, JTa_P = JT(Y, P, a_P, a_YP)

ginvJT = ginv(Y, P, JTa_Y, JTa_P)
ginvJT_Y = mat_spfy(ginvJT[0])
ginvJT_P = mat_spfy(ginvJT[1])
display(Math('\\text{ Formula for } g^{-1}J^T_Y: %s' % sprint(ginvJT_Y)))
display(Math('\\text{ Formula for } g^{-1}J^T_P: %s' % sprint(ginvJT_P)))

b_P, b_YP = J(Y, P, *ginvJT)

display(Math('\\text{ Formula for } (Jg^{-1}J^Ta)_P: %s' % sprint(b_P)))
display(Math('\\text{ Formula for } (Jg^{-1}J^Ta)_{YP}: %s' % sprint(b_YP)))

def DJ(Y, P, xi_Y, xi_P, eta_Y, eta_P):
    expr_P, expr_YP = J(Y, P, eta_Y, eta_P)
    der_P = DDR(expr_P, Y, xi_Y)+DDR(expr_P, P, xi_P)
    der_YP = DDR(expr_YP, Y, xi_Y)+DDR(expr_YP, P, xi_P)
    return mat_spfy(der_P), mat_spfy(der_YP)

dj_expr = DJ(Y, P, xi_Y, xi_P, eta_Y, eta_P)
display(Math('\\text{ Formula for } (DJ)\eta: %s' % sprint(dj_expr)))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Test for real positive-semidefinite
We have a separate notebook for complex-semidefnite. Adjusting the ratio between alpha and beta at different periods in optimization

In [9]:
import sys
!git clone https://github.com/pymanopt/pymanopt.git

sys.path.append("/content/pymanopt")

Cloning into 'pymanopt'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 4124 (delta 43), reused 33 (delta 12), pack-reused 4047
Receiving objects: 100% (4124/4124), 907.23 KiB | 1.23 MiB/s, done.
Resolving deltas: 100% (2863/2863), done.


In [13]:
import numpy as np
from numpy.random import (randint, randn)
from numpy import zeros, trace, allclose
import numpy.linalg as la

from ManNullRange.manifolds.RealPositiveSemidefinite import (
    RealPositiveSemidefinite, psd_ambient, psd_point)
from ManNullRange.manifolds.tools import (sym, extended_lyapunov)
from ManNullRange.tests.test_tools import check_zero, make_sym_pos, random_orthogonal


def solve_dist_with_man(man, A, X0, maxiter):
    from pymanopt import Problem
    from pymanopt.solvers import TrustRegions
    from pymanopt.function import Callable

    @Callable
    def cost(S):
        if not(S.P.dtype == np.float):
            raise(ValueError("Non real"))
        diff = (A - S.Y @ S.P @ S.Y.T)
        val = trace(diff @ diff.T)
        # print('val=%f' % val)
        return val

    @Callable
    def egrad(S):
        return psd_ambient(-4*A @ S.Y @ S.P,
                           2*(S.P-S.Y.T @ A @ S.Y))
    
    @Callable
    def ehess(S, xi):
        return psd_ambient(
            -4*A @ (xi.tY @ S.P + S.Y @ xi.tP),
            2*(xi.tP - xi.tY.T @ A @ S.Y - S.Y.T @ A @ xi.tY))

    prob = Problem(
        man, cost, egrad=egrad, ehess=ehess)

    solver = TrustRegions(maxtime=100000, maxiter=maxiter, use_rand=False)
    opt = solver.solve(prob, x=X0, Delta_bar=250)
    return opt

n, d = (1000, 50)
# simple function. Distance to a given matrix
# || S - A||_F^2
Y0, _ = np.linalg.qr(randn(n, d))
P0 = np.diag(randint(1, 1000, d)*.001)
A0 = sym(Y0 @ P0 @ Y0.T)
A = sym(randn(n, n))*1e-2 + A0

alpha = np.array([1, 1])
print("alpha %s" % str(alpha))

beta = alpha[1] * .1
man = RealPositiveSemidefinite(n, d, alpha=alpha, beta=beta)
XInit = man.rand()
opt_pre = solve_dist_with_man(man, A, X0=XInit, maxiter=20)

beta = alpha[1] * 1
man = RealPositiveSemidefinite(n, d, alpha=alpha, beta=beta)
opt_mid = solve_dist_with_man(man, A, X0=opt_pre, maxiter=20)
# opt_mid = opt_pre

beta = alpha[1] * 30
man = RealPositiveSemidefinite(n, d, alpha=alpha, beta=beta)
opt = solve_dist_with_man(man, A, X0=opt_mid, maxiter=50)
opt_mat = opt.Y @ opt.P @ opt.Y.T
if False:
    print(A0)
    print(opt_mat)
print(np.max(np.abs(A0-opt_mat)))


alpha [1 1]
Optimizing...
                                            f: +2.757896e+05   |grad|: 4.792588e+05
acc       k:     1     num_inner:     3     f: +1.130920e+05   |grad|: 1.884589e+05   reached target residual-kappa (linear)
acc       k:     2     num_inner:     3     f: +4.615305e+04   |grad|: 7.406179e+04   reached target residual-kappa (linear)
acc       k:     3     num_inner:     3     f: +1.872130e+04   |grad|: 2.908329e+04   reached target residual-kappa (linear)
acc       k:     4     num_inner:     3     f: +7.558696e+03   |grad|: 1.141192e+04   reached target residual-kappa (linear)
acc       k:     5     num_inner:     3     f: +3.050542e+03   |grad|: 4.472914e+03   reached target residual-kappa (linear)
acc       k:     6     num_inner:     2     f: +1.275048e+03   |grad|: 1.758553e+03   reached target residual-kappa (linear)
acc       k:     7     num_inner:     2     f: +5.477706e+02   |grad|: 6.898368e+02   reached target residual-kappa (linear)
acc       k:   